In [100]:
import pandas as pd

from pulp import LpProblem, LpMaximize, lpSum, LpVariable, LpStatus, LpInteger, LpBinary

data = pd.read_excel("data_2022.xlsx").dropna()

#PLAYED_AT_LEAST = 30
#data = initial_data[initial_data['PLD'] >= PLAYED_AT_LEAST]

In [101]:
data

,Code,Name,Club,Value,Pts,Position
0,145,A. Ramsdale,ARS,3.2,19,GK
1,124,B. Leno,ARS,2.8,-3,GK
2,101,E. Martinez,AV,3.1,10,GK
3,121,R. Olsen,AV,2.6,-2,GK
4,149,R. Sanchez,BHA,2.8,17,GK
...,...,...,...,...,...,...
459,651,L. Delap,MC,5.5,2,ST
460,653,S. Surridge,NOT,5.5,0,ST
461,626,L. Grabban,NOT,5.5,0,ST
462,640,T. Awoniyi,NOT,5.5,0,ST


In [115]:
# Create the data groups

PLAYERS = data['Name'].tolist()

TEAMS = {
    team: data[data['Club'] == team]['Name'].tolist() for team in data['Club'].unique().tolist()
}

POSITIONS = {
    position: data[data['Position'] == position]['Name'].tolist() for position in data['Position'].unique().tolist()
}

values = {
    player: data[data['Name'] == player].iloc[0]['Value'] for player in PLAYERS
}

points = {
    player: data[data['Name'] == player].iloc[0]['Pts'] for player in PLAYERS
}

codes = {
     player: data[data['Name'] == player].iloc[0]['Code'] for player in PLAYERS
}


In [111]:
# Disallowed teams for defensive players
DISALLOWED_DF_TEAMS = ['BHA', 'BRE', 'SOT', 'WAT', 'WOL', 'CP', 'NOR', 'LEE', 'NEW']
ALLOWED_DF_TEAMS = ["MC", "LIV", "CHE", "TOT"]
DF_POSITIONS = ['GK', 'FB', "CB"]

# Disallowed players
DISALLOWED_PLAYERS = ["M. Antonio", "D. James", "C. Gallagher",
                      "M. Alonso", 'H. Kane', "T. Chalobah"]

MUST_HAVE_PLAYERS = ["E. Haaland"]

df_players = POSITIONS['FB'] + POSITIONS["CB"]
gk_players = POSITIONS['GK']
COMBINED_DF_PLAYERS = df_players + gk_players

ONE_OF_TEAMS = ["WH"]


In [112]:
# Create problem

model = LpProblem("Fantasy", LpMaximize)

# variables
position_constraints = {
    'GK': 1,
    'FB': 2,
    "CB": 2,
    'MF': 4,
    'ST': 2
}

# Create player variables

player_vars = LpVariable.dicts("PlayerVariables", [(i) for i in PLAYERS], 0, 1, LpBinary)

## CONSTRAINTS

# Value constraint
model += lpSum(player_vars[(i)] * values[i] for i in PLAYERS) <= 50.0

# Team constraint
for team, players in TEAMS.items():
    model += lpSum(player_vars[(i)] for i in players) <= 2

# Disallowed defensive combinations
for team in TEAMS:
    if team not in ALLOWED_DF_TEAMS:
        model += lpSum(player_vars[(i)] for i in TEAMS[team] if i in COMBINED_DF_PLAYERS) == 0

# Only one defensive player per team to reduce correlation
#for team in TEAMS:
#    if team in TEAMS.keys():
#        model += lpSum(player_vars[(i)] for i in TEAMS[team] if i in COMBINED_DF_PLAYERS) <= 1

# Disallowed players
for player in DISALLOWED_PLAYERS:
    if player in PLAYERS:
        model += player_vars[(player)] == 0

# Must have players
for player in MUST_HAVE_PLAYERS:
    if player in PLAYERS:
        model += player_vars[(player)] == 1
    
# One of Teams
for team in ONE_OF_TEAMS:
    model += lpSum(player_vars[(i)] for i in TEAMS[team]) <= 1 

# Position constraint
for position, players in POSITIONS.items():
    model += lpSum(player_vars[(i)] for i in players) == position_constraints[position]
    
## OBJECTIVE FUNCTION
model += lpSum(player_vars[(i)] * points[i] for i in PLAYERS)


## SOLVE
model.solve()
    

1

In [113]:
LpStatus[model.status]

'Optimal'

In [116]:
total_sum = 0
for i in PLAYERS:
    if player_vars[(i)].varValue == 1:
        print(f'Code: {codes[i]}, Player: {i}, Points: {points[i]}')
        total_sum += points[i]
total_sum

Code: 144, Player: E. Mendy, Points: 31
Code: 232, Player: J. Cancelo, Points: 70
Code: 239, Player: A. Robertson, Points: 70
Code: 307, Player: E. Dier, Points: 38
Code: 375, Player: V. Van Dijk, Points: 70
Code: 549, Player: M. Mount, Points: 55
Code: 502, Player: D. Kulusevski, Points: 60
Code: 509, Player: S. Benrahma, Points: 36
Code: 529, Player: E. Smith Rowe, Points: 34
Code: 646, Player: E. Haaland, Points: 70
Code: 695, Player: Y. Wissa, Points: 25


559